In [1]:
# This file is made to be run once initially, to resize the dataset & create 3 new datasets
# 1. MAX_SIZE 
# 2. MIN_SIZE
# 3. MEAN_SIZE which is the mean of all the sizes rounded to nearest integer
# 4. FIXED_SIZE_interpolation which is 256x256
# 5. FIXED_SIZE_MIDDLE_CROP which is 256x256 
#    (this interpolates if image is smaller otherwise crops the middle)

In [2]:
import cv2
import os
from tqdm import tqdm
import shutil

In [3]:
# Constants
DATASET_PATH = 'dataset'
FLOODED_PATH = 'flooded'
NON_FLOODED_PATH = 'non-flooded'
FIXED_SIZE = 224 # ! Change this if need another fixed size

In [4]:
# Move the FLOODED_PATH and NON_FLOODED_PATH to a new normal directoy

path = os.getcwd()

new_dataset_path = os.path.join(path, 'dataset')
os.mkdir(new_dataset_path)
new_flooded_path = os.path.join(new_dataset_path, FLOODED_PATH)
os.mkdir(new_flooded_path)
new_non_flooded_path = os.path.join(new_dataset_path, NON_FLOODED_PATH)
os.mkdir(new_non_flooded_path)

for image in tqdm(os.listdir(os.path.join(path, os.path.join(DATASET_PATH, FLOODED_PATH)))):
    shutil.move(os.path.join(path, os.path.join(DATASET_PATH, FLOODED_PATH), image), new_flooded_path)
for image in tqdm(os.listdir(os.path.join(path, os.path.join(DATASET_PATH, NON_FLOODED_PATH)))):
    shutil.move(os.path.join(path, os.path.join(DATASET_PATH, NON_FLOODED_PATH), image), new_non_flooded_path)

# Remove the old FLOODED_PATH and NON_FLOODED_PATH
os.rmdir(os.path.join(path, os.path.join(DATASET_PATH, FLOODED_PATH)))
os.rmdir(os.path.join(path, os.path.join(DATASET_PATH, NON_FLOODED_PATH)))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'd:\\Fourth_Year\\Second_Term\\BD\\Flooding-Detection\\dataset'

In [5]:
# Now for the resizig part
# Create three new folders in the dataset folder
# 1. max_size
# 2. min_size
# 3. mean_size
# 4. fixed_size_interpolation
# 5. fixed_size_middle_crop

# Create the folders
max_size_path = os.path.join(path, DATASET_PATH, 'max_size')
min_size_path = os.path.join(path, DATASET_PATH, 'min_size')
mean_size_path = os.path.join(path, DATASET_PATH, 'mean_size')
fixed_size_interpolation_path = os.path.join(path, DATASET_PATH, 'fixed_size_interpolation')
fixed_size_middle_crop_path = os.path.join(path, DATASET_PATH, 'fixed_size_middle_crop')

os.mkdir(max_size_path)
os.mkdir(min_size_path)
os.mkdir(mean_size_path)
os.mkdir(fixed_size_interpolation_path)
os.mkdir(fixed_size_middle_crop_path)

os.mkdir(os.path.join(max_size_path, FLOODED_PATH))
os.mkdir(os.path.join(max_size_path, NON_FLOODED_PATH))

os.mkdir(os.path.join(min_size_path, FLOODED_PATH))
os.mkdir(os.path.join(min_size_path, NON_FLOODED_PATH))

os.mkdir(os.path.join(mean_size_path, FLOODED_PATH))
os.mkdir(os.path.join(mean_size_path, NON_FLOODED_PATH))

os.mkdir(os.path.join(fixed_size_interpolation_path, FLOODED_PATH))
os.mkdir(os.path.join(fixed_size_interpolation_path, NON_FLOODED_PATH))

os.mkdir(os.path.join(fixed_size_middle_crop_path, FLOODED_PATH))
os.mkdir(os.path.join(fixed_size_middle_crop_path, NON_FLOODED_PATH))

In [6]:
images_paths = []

folder_path = os.path.join(path, os.path.join(new_dataset_path, FLOODED_PATH))
for image in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image)
    images_paths.append(image_path)

folder_path = os.path.join(path, os.path.join(new_dataset_path, NON_FLOODED_PATH))
for image in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image)
    images_paths.append(image_path)

In [7]:
image_sizes = []

for image_path in tqdm(images_paths):
    image = cv2.imread(image_path)
    image_sizes.append((image.shape[0], image.shape[1]))

100%|██████████| 922/922 [00:58<00:00, 15.75it/s]


In [8]:
max_width = max(image_sizes, key=lambda x: x[0])[0]
max_height = max(image_sizes, key=lambda x: x[1])[1]

min_width = min(image_sizes, key=lambda x: x[0])[0]
min_height = min(image_sizes, key=lambda x: x[1])[1]

mean_width = int(round(sum([x[0] for x in image_sizes]) / len(image_sizes)))
mean_height = int(round(sum([x[1] for x in image_sizes]) / len(image_sizes)))

print('Max dimensions: ', (max_width, max_height))
print('Min dimensions: ', (min_width, min_height))
print('Mean dimensions: ', (mean_width, mean_height))

Max dimensions:  (3072, 4720)
Min dimensions:  (152, 123)
Mean dimensions:  (1352, 1794)


In [9]:
# Resize images

for image_path in tqdm(images_paths):
    # To shrink an image, it will generally look best with INTER_AREA interpolation,
    # To enlarge an image, it will generally look best with INTER_CUBIC
    # ! From the docs
    image = cv2.imread(image_path)
    max_image = cv2.resize(image, (max_width, max_height), interpolation=cv2.INTER_CUBIC)
    min_image = cv2.resize(image, (min_width, min_height), interpolation=cv2.INTER_AREA)

    if image.shape[0] * image.shape[1] > mean_width * mean_height:
        mean_image = cv2.resize(image, (mean_width, mean_height), interpolation=cv2.INTER_AREA)
    else:
        mean_image = cv2.resize(image, (mean_width, mean_height), interpolation=cv2.INTER_CUBIC)

    if image.shape[0] * image.shape[1] > FIXED_SIZE * FIXED_SIZE:
        fixed_size_interpolation_image = cv2.resize(image, (FIXED_SIZE, FIXED_SIZE), interpolation=cv2.INTER_AREA)
    else: 
        fixed_size_interpolation_image = cv2.resize(image, (FIXED_SIZE, FIXED_SIZE), interpolation=cv2.INTER_CUBIC)
    
    if image.shape[0] > FIXED_SIZE and image.shape[1] > FIXED_SIZE:
        fixed_size_middle_crop_image = image[image.shape[0] // 2 - FIXED_SIZE // 2: image.shape[0] // 2 + FIXED_SIZE // 2,
                                             image.shape[1] // 2 - FIXED_SIZE // 2: image.shape[1] // 2 + FIXED_SIZE // 2]
    else:
        fixed_size_middle_crop_image = cv2.resize(image, (FIXED_SIZE, FIXED_SIZE), interpolation=cv2.INTER_CUBIC)

    # Save the images
    if 'non-flooded' not in image_path:
        name = image_path.split(FLOODED_PATH)[-1]
        while name[0] == '\\':
            name = name[1:]
        cv2.imwrite(os.path.join(max_size_path, FLOODED_PATH, name), max_image)
        cv2.imwrite(os.path.join(min_size_path, FLOODED_PATH, name), min_image)
        cv2.imwrite(os.path.join(mean_size_path, FLOODED_PATH, name), mean_image)
        cv2.imwrite(os.path.join(fixed_size_interpolation_path, FLOODED_PATH, name), fixed_size_interpolation_image)
        cv2.imwrite(os.path.join(fixed_size_middle_crop_path, FLOODED_PATH, name), fixed_size_middle_crop_image)

    else:
        name = image_path.split(NON_FLOODED_PATH)[-1]
        while name[0] == '\\':
            name = name[1:]
        cv2.imwrite(os.path.join(max_size_path, NON_FLOODED_PATH, name), max_image)
        cv2.imwrite(os.path.join(min_size_path, NON_FLOODED_PATH, name), min_image)
        cv2.imwrite(os.path.join(mean_size_path, NON_FLOODED_PATH, name), mean_image)
        cv2.imwrite(os.path.join(fixed_size_interpolation_path, NON_FLOODED_PATH, name), fixed_size_interpolation_image)
        cv2.imwrite(os.path.join(fixed_size_middle_crop_path, NON_FLOODED_PATH, name), fixed_size_middle_crop_image)

100%|██████████| 922/922 [05:19<00:00,  2.88it/s]
